# Libs and pre-definitions

In [1]:
import os
REPOSITORY_PATH = r'/home/luis-felipe/UncEst'
DATA_PATH = os.path.join(REPOSITORY_PATH,'data')
#CORRUPTED_DATA_PATH = os.path.join(DATA_PATH,'corrupted')

PATH_MODELS = os.path.join(REPOSITORY_PATH,'torch_models')
PATH_TRAINER = os.path.join(PATH_MODELS,'trainer')

PATH_FIGS = os.path.join(REPOSITORY_PATH,'figs','EvalMIMO')

### Bibliotecas padrões python e utils pytorch

In [2]:
import torch
from torch import nn
import numpy as np
import pandas as pd
import matplotlib
from matplotlib import pyplot as plt
import os

In [3]:
# Define o computador utilizado como cuda (gpu) se existir ou cpu caso contrário
print(torch.cuda.is_available())
dev = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

True


### Bibliotecas desenvolvidas

https://github.com/lfpc/Uncertainty_Estimation

In [4]:
import NN_models as models
from NN_models import vgg
from NN_models import wide_resnet
import NN_utils as utils
import NN_utils.train_and_eval as TE
import torch_data
from NN_models import mimo


# Train classifier

## General definitions

In [5]:
loss_criterion = nn.NLLLoss().cuda()

#risk_dict = {'selective_risk_mcp':  lambda x,label: unc_comp.selective_risk(x,label,unc_type = unc.MCP_unc)}

## CIFAR 10

In [6]:
data = torch_data.Cifar10(data_dir = DATA_PATH)
num_classes = data.n_classes

Files already downloaded and verified
Files already downloaded and verified


In [ ]:
ens = 6
model = models.VGG_16
#model = models.Model_CNN
model = mimo.MIMOModel(model,num_classes,n_ensembles = ens,name = f'MIMO_VGG_10_{ens}').to(dev)
optimizer = torch.optim.SGD(model.parameters(), lr=0.1,momentum = 0.9,weight_decay = 5e-4,nesterov = True)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 25, gamma=0.5)

model_trainer = mimo.Trainer_MIMO(model,optimizer,loss_criterion,None, data.test_dataloader,lr_scheduler = scheduler)
model_trainer.fit(data.train_dataloader,200, checkpoint = True, PATH = PATH_MODELS)

print(f'Max acc = {max(model_trainer.val_acc)} - epoch = {np.argmax(model_trainer.val_acc)}')
torch.save(model.state_dict(), PATH_MODELS+f'/MIMO_VGG_10_{ens}.pt')



/home/luis-felipe/anaconda3/lib/python3.8/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


Progress::   0%|          | 0/200 [00:00<?, ?it/s]

/home/luis-felipe/anaconda3/lib/python3.8/site-packages/torch/optim/lr_scheduler.py:154: UserWarning: The epoch parameter in `scheduler.step()` was not necessary and is being deprecated where possible. Please use `scheduler.step()` to step the scheduler. During the deprecation, if epoch is different from None, the closed form is used instead of the new chainable form, where available. Please open an issue if you are unable to replicate your use case: https://github.com/pytorch/pytorch/issues/new/choose.
  warnings.warn(EPOCH_DEPRECATION_WARNING, UserWarning)


## CIFAR 100

In [14]:
data = torch_data.Cifar100(data_dir = DATA_PATH)
num_classes = data.n_classes

Files already downloaded and verified
Files already downloaded and verified


In [16]:
ens = 6
model_2 = models.VGG_16
model_2 = mimo.MIMOModel(model_2,num_classes,n_ensembles = ens,name = f'MIMO_VGG_100_{ens}').to(dev)
optimizer = torch.optim.SGD(model_2.parameters(), lr=0.1,momentum = 0.9,weight_decay = 5e-4,nesterov = True)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 25, gamma=0.5)

model_trainer_2 = mimo.Trainer_MIMO(model_2,optimizer,loss_criterion,None, data.test_dataloader,lr_scheduler = scheduler)
model_trainer_2.fit(data.train_dataloader,200, checkpoint = True, PATH = PATH_MODELS)

print(f'Max acc = {max(model_trainer_2.val_acc)} - epoch = {np.argmax(model_trainer_2.val_acc)}')
torch.save(model_2.state_dict(), PATH_MODELS+f'/MIMO_VGG_100_{ens}.pt')


Progress::   0%|          | 0/200 [00:00<?, ?it/s]

Max acc = 70.48 - epoch = 183
